In [1]:
import os
os.chdir('../')
%pwd

'/Users/tonyzhang/Desktop/Chicken-Disease-Classification/chicken-disease-classification'

In [2]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_chkpt_filepath = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_chkpt_filepath),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config =PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

config/config.yaml params.yaml


In [4]:
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf 
import time 

2024-08-22 23:05:17.255882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config 

    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
        ]


In [6]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
    tb_ckpt_callbacks = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-08-22 23:05:32,309: INFO: common: Successully read the yaml file at config/config.yaml]
[2024-08-22 23:05:32,314: INFO: common: Successully read the yaml file at params.yaml]
[2024-08-22 23:05:32,315: INFO: common: Created directory at artifacts]
[2024-08-22 23:05:32,317: INFO: common: Created directory at artifacts/prepare_callbacks/checkpoint_dir]
[2024-08-22 23:05:32,318: INFO: common: Created directory at artifacts/prepare_callbacks/tensorboard_logs]
